In [1]:
import pandas as pd
from pytz import timezone

sp = timezone('America/Sao_Paulo')

In [2]:
from pymongo import MongoClient
from pymongo.database import Database

database = MongoClient('mongodb://172.30.1.20:50102')

database['admin'].authenticate('worker', 'worker@sofie')

coll_transaction = database['config']['transaction']
coll_ledger = database['config']['ledger']
coll_sofier = database['config']['sofier']
coll_consumer = database['config']['consumer']

In [3]:
dict_sofier = {item['sofier_id']: item['short_name'] for item in coll_sofier.find(projection={'sofier_id': 1, 'short_name': 1})}
dict_consumer = {item['name']: item['full_name'] for item in coll_consumer.find(projection={'name': 1, 'full_name': 1})}

In [4]:
rows = list()
for item in coll_transaction.find({'status.status': 'FINISHED'}):
    rows.append(
        dict(
            transaction=item['transaction'][:9],
            date=sp.localize(item['when']).strftime('%d/%m %H:%M'),
            sofier='{} - {}'.format(item['actors']['sofier'][:3] if item['actors']['sofier'] else '---', dict_sofier.get(item['actors']['sofier'], '---').split()[0]),
            consumer='{} - {}'.format(item['actors']['consumer'][:3], dict_consumer.get(item['actors']['consumer'], 'DESCONHECIDO').split()[0]) if item['actors']['consumer'] else '---',
            value=item['payment_condition']['price'] if item['payment_condition'] else '---',
            status=item['status']['status'],
            success=item['status']['success'],
            fraud=item['fraud_prevention']['log'][0]['Rating'] if item['fraud_prevention']['when'] else '---',
            authorization=(item['payment_info']['credit_card']['authorization']['log'][0]['Payment']['ReturnMessage'] if isinstance(item['payment_info']['credit_card']['authorization']['log'][0], dict) else item['payment_info']['credit_card']['authorization']['log'][0][0]['Message']) if item['payment_info']['credit_card']['authorization']['when'] else '---',
            capture=(item['payment_info']['credit_card']['catch']['log'][0]['ReturnMessage'] if isinstance(item['payment_info']['credit_card']['catch']['log'][0], dict) else item['payment_info']['credit_card']['catch']['log'][0][0]['Message']) if item['payment_info']['credit_card']['catch']['when'] else '---',
            cancel=(item['payment_info']['credit_card']['cancellation']['log'][0]['ReturnMessage'] if isinstance(item['payment_info']['credit_card']['cancellation']['log'][0], dict) else item['payment_info']['credit_card']['catch']['log'][0][0]['Message']) if item['payment_info']['credit_card']['cancellation']['when'] else '---',
            reward=coll_ledger.count_documents({'transaction': item['transaction'], 'type': 'Cr'}),
            reversal=coll_ledger.count_documents({'transaction': item['transaction'], 'type': 'Dr'}),
            payment_id=(item['payment_info']['credit_card']['authorization']['log'][0]['Payment']['PaymentId'] if isinstance(item['payment_info']['credit_card']['authorization']['log'][0], dict) else '---') if item['payment_info']['credit_card']['authorization']['when'] else '---',
        )
    )
planilha = pd.DataFrame(rows)

In [5]:
planilha

,authorization,cancel,capture,consumer,date,fraud,payment_id,reversal,reward,sofier,status,success,transaction,value
0,<![CDATA[\nO XML informado não é valido:\n- cv...,---,---,323 - Thiago,19/09 23:08,1,bf65ae54-96ae-4858-9d6d-20c7847f1f03,0,0,003 - Mario,FINISHED,False,fb2c9de2-,3.50
1,<![CDATA[\nO XML informado não é valido:\n- cv...,---,Transaction not available to capture,196 - JOSÉ,19/09 23:31,5,96d40715-0ba0-48ba-9226-7bddf5cd1a1e,1,1,003 - Thiago,FINISHED,False,11266814-,3.50
2,Transacao autorizada,---,Transacao capturada com sucesso,196 - JOSÉ,19/09 23:47,5,e813af1d-4964-4ec2-a597-f08f897d5696,0,1,003 - Thiago,FINISHED,True,6c38bf0c-,3.50
3,Transacao autorizada,Transacao desfeita,---,323 - Thiago,20/09 15:22,1,7de69248-2c38-4bd6-9f2a-7e217cc70110,0,0,003 - Mario,FINISHED,False,0367f9c4-,3.50
4,Transacao autorizada,---,Transacao capturada com sucesso,355 - TAMIRES,20/09 15:29,5,0dd8882a-282b-4496-a51f-cb2a2aadff8f,0,1,003 - Mario,FINISHED,True,f4fc396c-,3.50
5,Transacao autorizada,---,Transacao capturada com sucesso,356 - JORGE,20/09 18:57,5,14113ecd-3049-4980-a162-c69024f38545,0,1,003 - Mario,FINISHED,True,04768218-,3.50
6,Transacao autorizada,Transacao desfeita,---,266 - Fransisco,20/09 20:05,---,a24aaa41-a755-4d63-9107-41f5cf8f41c8,0,0,003 - Japa,FINISHED,False,92ed225a-,3.50
7,Transacao autorizada,---,Transacao capturada com sucesso,281 - Erik,20/09 20:23,5,13da76d6-2f50-4b5a-b894-a0ae5c5f021a,0,1,003 - Thiago,FINISHED,True,f960fc1c-,3.50
8,Transacao autorizada,---,Transacao capturada com sucesso,281 - Erik,20/09 20:41,5,6b614330-e6e7-4d87-9f9a-47795a0d7132,0,1,003 - Mario,FINISHED,True,86bcff0a-,3.50
9,Transacao autorizada,---,Transacao capturada com sucesso,027 - Elison,20/09 21:54,5,09be8964-f4b9-42b1-acd9-5d1b0d65abe1,0,1,003 - Mario,FINISHED,True,d17aa1be-,3.50
